Imports

In [1]:
import torch
import os
import sys
from torchvision import transforms
from IPython.display import Image 
import progressbar


script_root_dir = '../'
if script_root_dir not in sys.path:
    sys.path.append(script_root_dir)
    
from train import dataLoader
from train.configParser import ConfigParser
from myhelpers.color_PCA import Color_PCA

Parameters

In [2]:
experimentsPath="/home/elhamod/HGNN/experiments/"
experimentName="NoEarlyStopping"
dataPath="/data/BGNN_data/"

imageDimension = 224

applyHorizontalFlip = True
applyHorizontalFlipProbability = 0.1

applyTranslation=True
applyTranslationAmount = 0.1

applyColorPCA=True
colorPCAperturbationMagnitude = 0.1

numOfAugmentedVersions = 10

cuda = 7

In [3]:
# set cuda
if torch.cuda.is_available():
    torch.cuda.set_device(cuda)

Load dataset

In [4]:
# Get the dataset
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
experiment_params = next(iter(config_parser.getExperiments()))
experimentPathAndName = os.path.join(experimentsPath, experimentName)
datasetManager = dataLoader.datasetManager(experimentPathAndName)
datasetManager.updateParams(config_parser.fixPaths(experiment_params))
dataset = datasetManager.getDataset()

# disable normalization before augmentation.
dataset.toggle_image_loading(augmentation=False, normalization=False)

# Create appropriate loaders
loaderPerImage = torch.utils.data.DataLoader(dataset, batch_size=1)
datasetSize = len(dataset)
loaderAllImages = torch.utils.data.DataLoader(dataset, batch_size=datasetSize)
print("number of images", datasetSize)

Creating dataset...


Loading images: 100%|██████████| 2600/2600 [02:32<00:00, 17.02it/s, fileName=/data/BGNN_data/INHS_cropped/images/INHS_FISH_63588.jpg]  

Creating dataset... Done.
number of images 2600


Generate data

In [5]:
color_pca = Color_PCA(loaderAllImages, colorPCAperturbationMagnitude)

with progressbar.ProgressBar(max_value=datasetSize) as bar:
    k=1
    for batch in loaderPerImage:
        original = batch['image'].squeeze()
        fileName = batch['fileName'][0]
        prefix, ext = os.path.splitext(fileName)

        if torch.cuda.is_available():
            original = original.cpu()
        
        # Generate a scaled version
        savedFileName = os.path.join(dataset.csv_processor.get_image_full_path(), prefix+"_"+str(imageDimension)+ext)
        scaled = transforms.ToPILImage()(original)
        scaled.save(savedFileName)
        print(savedFileName, "saved!")

        # Generate augmented versions
        for j in range(numOfAugmentedVersions):
            image = transforms.ToPILImage()(original)

            if applyHorizontalFlip:
                image = transforms.RandomHorizontalFlip(p=applyHorizontalFlipProbability)(image)
    #             display(image)

            if applyTranslation:
                RGBmean = (int(0.485*255), int(0.456*255), int(0.406*255))
                image = transforms.RandomAffine(degrees=0, translate=(applyTranslationAmount,
                                                                      applyTranslationAmount),
                                               fillcolor=RGBmean)(image)
    #             display(image)

            if applyColorPCA:
                image = transforms.ToTensor()(image).unsqueeze(0)
                image = transforms.Lambda(color_pca.perturb_color)(image).type(torch.float32) 

                image = transforms.ToPILImage()(image.squeeze())
    #             display(image)


            savedFileName = os.path.join(dataset.csv_processor.get_image_full_path(), prefix+"_"+str(imageDimension)+"_aug"+str(j)+ext)
            image.save(savedFileName)
            print(savedFileName, "saved!")
            
        bar.update(k)
        k = k + 1

    
    

Transforming images: 100%|██████████| 2600/2600 [00:53<00:00, 48.18it/s]
  0% (3 of 2600) |                       | Elapsed Time: 0:00:00 ETA:   0:01:32

/data/BGNN_data/INHS_cropped/images/INHS_FISH_107563_224.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_107563_224_aug0.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_107563_224_aug1.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_107563_224_aug2.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_107563_224_aug3.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_107563_224_aug4.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_107563_224_aug5.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_107563_224_aug6.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_107563_224_aug7.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_107563_224_aug8.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_107563_224_aug9.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_46357_224.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_46357_224_aug0.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_46357_224_aug1.jp

  0% (7 of 2600) |                       | Elapsed Time: 0:00:00 ETA:   0:01:59

/data/BGNN_data/INHS_cropped/images/INHS_FISH_012567_224_aug7.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_012567_224_aug8.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_012567_224_aug9.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_57501_224.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_57501_224_aug0.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_57501_224_aug1.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_57501_224_aug2.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_57501_224_aug3.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_57501_224_aug4.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_57501_224_aug5.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_57501_224_aug6.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_57501_224_aug7.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_57501_224_aug8.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_57501_224_aug9.jpg s

  0% (11 of 2600) |                      | Elapsed Time: 0:00:00 ETA:   0:02:08

...

/data/BGNN_data/INHS_cropped/images/INHS_FISH_21298_224.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_21298_224_aug0.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_21298_224_aug1.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_21298_224_aug2.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_21298_224_aug3.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_21298_224_aug4.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_21298_224_aug5.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_21298_224_aug6.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_21298_224_aug7.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_21298_224_aug8.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_21298_224_aug9.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_51174_224.JPG saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_51174_224_aug0.JPG saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_51174_224_aug1.JPG saved!
/d

 99% (2598 of 2600) |################### | Elapsed Time: 0:02:39 ETA:   0:00:00

/data/BGNN_data/INHS_cropped/images/INHS_FISH_68227_224_aug6.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_68227_224_aug7.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_68227_224_aug8.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_68227_224_aug9.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_64381_224.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_64381_224_aug0.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_64381_224_aug1.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_64381_224_aug2.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_64381_224_aug3.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_64381_224_aug4.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_64381_224_aug5.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_64381_224_aug6.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_64381_224_aug7.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_64381_224_aug8.jpg save

100% (2600 of 2600) |####################| Elapsed Time: 0:02:40 Time:  0:02:40


/data/BGNN_data/INHS_cropped/images/INHS_FISH_92277_224_aug0.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_92277_224_aug1.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_92277_224_aug2.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_92277_224_aug3.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_92277_224_aug4.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_92277_224_aug5.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_92277_224_aug6.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_92277_224_aug7.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_92277_224_aug8.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_92277_224_aug9.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_63588_224.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_63588_224_aug0.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_63588_224_aug1.jpg saved!
/data/BGNN_data/INHS_cropped/images/INHS_FISH_63588_224_aug2.jpg save